In [1]:
import os
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# submission_files = ['5933.csv', '5758.csv', '5679.csv', '5228.csv', '5122.csv', '5057.csv']
weights = [3, 2, 1]
submission_files = [f'{i}.csv' for i in range(1, 4)]

submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
a = []
for s in submission_files:
    a.append(s.rstrip('.csv'))
    
out = '_'.join(a)
print(out)

1_2_3


In [4]:
image_ids = submission_df[0]['image_id'].tolist()

In [5]:
annotation = '../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
iou_thr = 0.6

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
#     print(coco.getImgIds(imgIds=image_id))
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = predict_string.split()
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
        
    if len(boxes_list):
#         boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
#         boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
#         boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]  * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [7]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'ensemble({out}).csv', index=None)

submission.head()

,PredictionString,image_id
0,8 0.9914937615394592 173.24806213378906 234.92...,batch_01_vt/0021.jpg
1,8 0.9605919122695923 157.6902618408203 54.6061...,batch_01_vt/0028.jpg
2,1 0.9589161276817322 71.5116958618164 169.1573...,batch_01_vt/0031.jpg
3,8 0.9679356217384338 31.79630470275879 125.742...,batch_01_vt/0032.jpg
4,8 0.9833965301513672 150.25686645507812 0.0517...,batch_01_vt/0070.jpg
